In [8]:
import torch
import os
from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image
import numpy as np
from controlnet_aux import PidiNetDetector, HEDdetector

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

import gradio as gr
import cv2
import numpy as np
from PIL import Image
from gradio_client import Client
import os
import uuid
import random

checkpoint = "lllyasviel/control_v11p_sd15_scribble"

processor = HEDdetector.from_pretrained('lllyasviel/Annotators')

controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()


/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/hed/__init__.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle modul

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def create_directory(directory_name='images'):
  try:
    os.mkdir(directory_name)
    os.mkdir(directory_name+"/ARENA IA - CIn - Colagens")
    print(f"Directory '{directory_name}' created successfully.")
  except FileExistsError:
    print(f"Directory '{directory_name}' already exists.")
  except PermissionError:
    print(f"Permission denied: Unable to create '{directory_name}'.")
  except Exception as e:
    print(f"An error occurred: {e}")

In [3]:
def make_collage(path_img1, path_img2, uuid):
    # Carregar as duas imagens
    imagem1 = Image.open(path_img1)  # Imagem 1920x1440
    imagem2 = Image.open(path_img2)  # Imagem 704x512

    # Redimensionar a primeira imagem para 704px de largura, mantendo o aspect ratio
    nova_largura = 704
    proporcao = nova_largura / imagem1.width
    nova_altura = int(imagem1.height * proporcao)
    imagem1 = imagem1.resize((nova_largura, nova_altura))

    # Calcular a altura total para a colagem (precisamos de 1024 de altura)
    altura_total = 1024

    # Criar uma nova imagem com 704x1024 e fundo branco
    imagem_final = Image.new("RGB", (nova_largura, altura_total), "white")

    # Colar a primeira imagem no topo da imagem final
    imagem_final.paste(imagem1, (0, 0))

    # Colar a segunda imagem logo abaixo da primeira
    imagem_final.paste(imagem2, (0, imagem1.height))

    # Salvar o resultado
    imagem_final.save(f"images/ARENA IA - CIn - Colagens/{uuid}_collage.jpg")

In [4]:
def create_image(image_path, prompt):
  image = load_image(image_path)
  print(type(image))

  control_image = processor(image, scribble=True)

  seed_input = random.randint(0, 1000000)
  generator = torch.manual_seed(seed_input)

  n_prompt = "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, hands, flesh, skin, human, glass, sky, plant, faceless, paint, pillow, complex background" # "complex background, faceless"

  image2 = pipe(prompt, num_inference_steps=22, generator=generator, image=control_image, negative_prompt=n_prompt, guidance_scale=12).images[0]

  image_uuid = str(uuid.uuid4())

  path_o_img = 'images/' + image_uuid + '_original.png'
  path_g_img = 'images/' + image_uuid + '_generated.png'

  image.save(path_o_img)
  image2.save(path_g_img)
  make_collage(path_o_img, path_g_img, image_uuid)

  return image2

In [5]:
def process_image(image, prompt_option):

  match prompt_option:
    case "A":
      return create_image(image, "a picture of a stuffed animal, best quality, extremely detailed, plush, felt, face, lifelike, simple background, plain background")
    case "B":
      return create_image(image, "a picture of a stuffed animal, best quality, extremely detailed, plush, felt, face, lifelike, simple background, plain background")
    case "C":
      return create_image(image, "a picture of a stuffed animal, best quality, extremely detailed, plush, felt, face, lifelike, simple background, plain background")
    case _:
      return create_image(image, "a picture of a stuffed animal, best quality, extremely detailed, plush, felt, face, lifelike, simple background, plain background")


In [9]:
create_directory()

image_input = gr.Image(type="filepath" ,label="Upload an Image")
prompts_options = gr.Radio(
        ["Enhance colors", "Apply vintage effect", "Increase sharpness", "Add blur effect"],
        label="Select a Prompt"
)

iface = gr.Interface(
    fn=process_image,
    inputs=[image_input, prompts_options],
    outputs=gr.Image(label="Imagem processada"),
    live=False,
    title="",
    css="""
        @import url('https://fonts.googleapis.com/css2?family=Nunito:wght@400;700&display=swap');

        .gradio-container {
            font-family: 'Nunito', sans-serif;
            background: linear-gradient(#000000 25%, #ff0571 75%, #f98404); # verde 1eff05, laranja f98404, roxo 52057b, rosa ff0571
            color: #ffffff;
            text-align: center;
        }
        h1 {
            width: 100%;
            margin-top: -30px;
            max-width: 300px;
            display: block;
            margin-left: auto;
            margin-right: auto;
        }
    """
)

# Inicia a interface
iface.launch(debug=True)

Directory 'images' already exists.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://539c95955d04891e6d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<class 'PIL.Image.Image'>


  0%|          | 0/22 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://539c95955d04891e6d.gradio.live


In [ ]:
# This is a test message